# 乱数の生成

In [74]:
from scipy.stats import bernoulli
from scipy import stats

import numpy as np



# 高さの定義

これは、事後分布のlogを取った関数となる

あ、でもこれ、大丈夫か？ 

だって、事後分布計算できなくね。。

In [121]:
mus = 5
sigmas = 1

def pgauss(x):
    return stats.norm.pdf(x=x, loc=mus, scale=sigmas) 



In [122]:
def h(theta, func):
    # 確率密度のlogを取って、マイナスをつけたものが高さとなる。
    return - np.log(func(theta))

    

# 高さを微分の定義

In [156]:
def delta_h(theta):
    if theta == mus:
        return 0
    return (theta-mus)/(theta**2 -2 * mus * theta + mus**2)

In [157]:
print(delta_h(5))
print(delta_h(4))
print(delta_h(6))
print(delta_h(30))
print(delta_h(50))
print(delta_h(-30))
print(delta_h(-50))

0
-1.0
1.0
0.04
0.022222222222222223
-0.02857142857142857
-0.01818181818181818


# ハミルトニアンの定義

In [143]:

def hamiltonian(theta,func, p):
    return h(theta,func) + 0.5 * p**2
    
    

In [144]:
# ハミルトニアンの低さが確率密度の高さとなる。

hamiltonian(5,pgauss,2)

2.9189385332046727

# リープフロッグ法の定義

In [145]:
def leap_flog_step1(p, eta,func, theta):
    return p - eta * 0.5 * func(theta)

def leap_flog_step2(loc,p,eta):
    return loc + eta * p


In [146]:
a = leap_flog_step1(0, 0.05,delta_h, 4)
a

0.025

In [147]:
b = leap_flog_step2(4,a,0.05)

In [148]:
leap_flog_step3(a, 0.05, delta_h, b)

0.025

# リープフロッグによる遷移

In [153]:
# リープフロッグ法の実行
# initial param
theta = 5.1
p = 0
eta = 0.05
L = 96


ret = []
ret.append([1, p, theta, hamiltonian(theta,pgauss,p)])
for _ in range(L):
    p = leap_flog_step1(p, eta,delta_h, theta)
    theta = leap_flog_step2(theta,p,eta)
    p = leap_flog_step1(p, eta, delta_h, theta)
    ret.append([p, theta, hamiltonian(theta,pgauss, p)])


/Users/kojisugano/anaconda/envs/stan3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


In [154]:
ret

[[1, 0, 5.1, 0.9239385332046727],
 [-0.5357142857144515, 5.087499999999997, 1.0662615561639448],
 [-1.3598901098900638, 5.046428571428554, 1.8446668948155256],
 [-1.3827709118068214, 4.95151098901099, 1.8761418225675532],
 [-0.5950028817983215, 4.908151480247872, 1.100170823169155],
 [-0.09131116898518238, 4.892010700831158, 0.9289382423628817],
 [0.3877679181803436, 4.899020363349353, 0.9992189558986798],
 [0.9965489771087283, 4.930787492649192, 1.4178886506797919],
 [20.229398153118517, 4.998675261060226, 205.53321422936872],
 [39.088244884084915, 6.953727307961044, 766.7729077892363],
 [39.06905087708812, 8.907499749468718, 771.7485838975036],
 [39.05838717276723, 10.860632395669857, 780.8712488406861],
 [39.05092176491083, 12.813338466745442, 793.9303128757599],
 [39.04516213441193, 14.76572457216094, 810.8659697941163],
 [39.040468664180125, 16.717854680186633, 831.6520944456057],
 [39.03650631502958, 18.66977143857895, 856.2746767663712],
 [39.03307710427667, 20.621505311689592, 

In [164]:
def h(theta):
    global lam, alpha
    lam, alpha = 5,5
    return lam * theta - (alpha-1)*np.log(theta) 

def dh_dtheta(theta):
    global lam, alpha
    lam, alpha = 5,5
    return lam - (alpha - 1)/theta


def hamiltonian(p, theta):
    return h(theta) + 0.5*p**2

vhamiltonian = np.vectorize(hamiltonian)  # vectorize

def leapfrog_nexthalf_p(p, theta, eps=0.01):
    """
    1/2ステップ後のpを計算
    """
    return p - 0.5 * eps* dh_dtheta(theta)

def leapfrog_next_theta(p, theta, eps=0.01):
    """
    1ステップ後のθを計算
    """
    return theta + eps*p


def move_one_step(theta, p, eps=0.01, L=100, stlide=1):
    """
    リープフロッグ法でL回移動した１ステップを実行
    """
    ret = []
    ret.append([1, p, theta, hamiltonian(p,theta)])
    for _ in range(L):
        p = leapfrog_nexthalf_p(p, theta, eps)
        theta = leapfrog_next_theta(p, theta, eps)
        p = leapfrog_nexthalf_p(p, theta, eps)
        ret.append([1, p, theta, hamiltonian(p,theta)])
    return ret[::stlide]


In [171]:
# リープフロッグ法の実行
# initial param
theta = 0.1
p = 1
eps = 0.05
L = 100

result = move_one_step(theta, p, eps=eps, L=100, stlide=1)


In [170]:
result

[[1, 0, 0.1, 9.710340371976182],
 [1, 1.4456521739130435, 0.14375000000000002, 9.522423501188463],
 [1, 2.3001932367149758, 0.24456521739130438, 9.501363621425842],
 [1, 2.726626821401022, 0.3737693236714976, 9.522559343879818],
 [1, 2.9375098891046387, 0.5172278995314066, 9.537708432470016],
 [1, 3.0306564363057213, 0.6675203125819615, 9.546782613715822],
 [1, 3.052372190702492, 0.8202935431619788, 9.552327804366346],
 [1, 3.0270803102745263, 0.9727575316522108, 9.5558769587202],
 [1, 2.9689279209341217, 1.1230015741894315, 9.558254060690835],
 [1, 2.8867368345461895, 1.269650323745623, 9.559910286989721],
 [1, 2.786336499527375, 1.4116752576440503, 9.561103332056865],
 [1, 2.6717619559766743, 1.5482839736983605, 9.56198702765381],
 [1, 2.5459141222060624, 1.6788514532417178, 9.56265702171575],
 [1, 2.4109456041623263, 1.8028753859189668, 9.563174974793483],
 [1, 2.2684973526274415, 1.9199460136579505, 9.563581916697043],
 [1, 2.1198499054812556, 2.029725121181711, 9.56390591439182],
